In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('./cl2.csv')

In [3]:
from sklearn.model_selection import train_test_split

# splitting data
X_train, X_test, y_train, y_test = train_test_split(
                df.drop('diagnosis', axis=1),
                df['diagnosis'],
                test_size=0.2,
                random_state=42)

print("Shape of training set:", X_train.shape)
print("Shape of test set:", X_test.shape)

Shape of training set: (228, 31)
Shape of test set: (57, 31)


In [4]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()
logreg.fit(X_train, y_train)
predictions1 = logreg.predict(X_test)

c:\Users\fafaf\anaconda3\envs\tf\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [5]:
from sklearn.metrics import confusion_matrix, classification_report

print("Confusion Matrix: \n", confusion_matrix(y_test, predictions1))
print('\n')
print(classification_report(y_test, predictions1))

Confusion Matrix: 
 [[38  3]
 [ 2 14]]


              precision    recall  f1-score   support

           0       0.95      0.93      0.94        41
           1       0.82      0.88      0.85        16

    accuracy                           0.91        57
   macro avg       0.89      0.90      0.89        57
weighted avg       0.91      0.91      0.91        57



In [16]:
model = LogisticRegression(warm_start=True,penalty='l2')

In [17]:
import utils
utils.set_initial_params(model, n_features=X_train.shape[1], n_classes=2)

In [18]:
import flwr as fl
import warnings
from sklearn.metrics import log_loss

class IrisClient(fl.client.NumPyClient):
        def get_parameters(self, config):  # type: ignore
            return utils.get_model_parameters(model)

        def fit(self, parameters, config):  # type: ignore
            utils.set_model_params(model, parameters)
            # Ignore convergence failure due to low local epochs
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                model.fit(X_train, y_train)
            accuracy = model.score(X_train, y_train)
            return (
                utils.get_model_parameters(model),
                len(X_train),
                {"train_accuracy": accuracy},
            )

        def evaluate(self, parameters, config):  # type: ignore
            utils.set_model_params(model, parameters)
            loss = log_loss(y_test, model.predict_proba(X_test))
            accuracy = model.score(X_test, y_test)
            print(accuracy)
            return loss, len(X_test), {"test_accuracy": accuracy}

    # Start Flower client
fl.client.start_client(
    server_address="127.0.0.1:8080", client=IrisClient().to_client()
)

INFO flwr 2024-05-01 19:24:11,690 | grpc.py:52 | Opened insecure gRPC connection (no certificates were passed)
DEBUG flwr 2024-05-01 19:24:11,702 | connection.py:55 | ChannelConnectivity.IDLE
DEBUG flwr 2024-05-01 19:24:11,704 | connection.py:55 | ChannelConnectivity.READY


0.9649122807017544
0.9824561403508771
0.9824561403508771
0.9824561403508771
0.9824561403508771
0.9824561403508771
0.9824561403508771
0.9824561403508771
0.9824561403508771
0.9824561403508771
0.9824561403508771
0.9824561403508771
0.9824561403508771
0.9824561403508771
0.9824561403508771
0.9824561403508771
0.9824561403508771
0.9824561403508771


DEBUG flwr 2024-05-01 19:24:12,347 | connection.py:220 | gRPC channel closed
INFO flwr 2024-05-01 19:24:12,348 | app.py:398 | Disconnect and shut down


0.9824561403508771
0.9824561403508771
0.9824561403508771
0.9824561403508771
0.9824561403508771
0.9824561403508771
0.9824561403508771


In [19]:
print(model.coef_)

[[-1.31967070e-02 -9.95126773e-01 -2.18727170e-01 -3.82155037e-01
   2.07680967e-02  4.17902429e-02  1.66831942e-01  2.75630735e-01
   1.17443553e-01  8.84330549e-02  1.19885944e-02 -2.26381002e-02
  -4.59366826e-01  1.48304957e-03  1.14120377e-01  4.88681734e-03
   1.91833926e-02  5.45228425e-02  1.77466670e-02  2.27397515e-02
   2.36365885e-04 -1.11616890e+00  4.44572059e-01  3.28493545e-01
   1.59877257e-02  8.33240430e-02  4.79810546e-01  7.74920686e-01
   2.24007421e-01  2.92702347e-01  3.50153642e-02]]


In [20]:
predictions1 = model.predict(X_test)
print(classification_report(y_test, predictions1))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99        41
           1       1.00      0.94      0.97        16

    accuracy                           0.98        57
   macro avg       0.99      0.97      0.98        57
weighted avg       0.98      0.98      0.98        57



In [21]:
print("Confusion Matrix: \n", confusion_matrix(y_test, predictions1))


Confusion Matrix: 
 [[41  0]
 [ 1 15]]
